In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
import tqdm
import statsmodels.formula.api as sm

sns.set()

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path), *args, **kwargs)
    except FileNotFoundException as e:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)


def followorbit(vals, path):
    out = []
    for n in range(100):
        closest05 = np.clip(round(path[n] * 2) / 2, 0, 10)
        rank = vals.query(f'nbopp == {n} and othercoop == {closest05}')['rank'].values[0]
        out.append(rank)
    return np.array(out)

import re
def gethighest(vals, prefix="logall_", which=1):
    if which == 1:
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
    else:
        trueval = [int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals]
        itrueval = sorted(list(enumerate(trueval)), key=lambda x: x[1])
        m = itrueval[-which][0]
    return vals[m]

    
def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

def sorted_nicely(l): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

import platform
hostname = platform.node()
print(hostname)

if hostname == 'cluster':
    prefix = '/home/ecoffet/robocoop/'
else:
    prefix = '/home/pecoffet/remoterobo/'

In [ ]:
import json
paths = sorted(glob.glob(f'{prefix}/logs/negociate17-3tau/**/rep00/', recursive=True))
errors = 0
allstat = []
for path in tqdm.tqdm_notebook(paths):
    try:
        fitness = read_csvx(path+'/../fit.txt', delimiter="\t", names=['gen', 'min', 'q1', 'med', 'q3', 'max'])
        params = readparams(sorted(glob.glob(path + '/properties*'))[0])
        coop = read_csvx(gethighest(glob.glob(path + '/coop_199.txt'), 'coop_'), delimiter="\t")
        params['coop'] = coop['coop'].mean()
    except Exception as e:
        errors+=1
        pass
    else:
        allstat.append(params)
df = pd.DataFrame(allstat)

In [ ]:
errors / (errors + len(df))


In [ ]:
df['nbRobots'] = df['gInitialNumberOfRobots']
df['varPheno'] = df['fakeRobots']
df['nbRep'] = df['nbEvaluationsPerGeneration']
df['norm_tau'] = df['tau'] / 100000
df['norm_tau'] = df['norm_tau'].replace(0, 1)

In [ ]:
g = sns.relplot(data=df, x='nbRobots', y='coop', row='tau', col='fakeRobots', aspect=1.5, kind="line")
g.map_dataframe(sns.scatterplot, x="nbRobots", y="coop")
#g.map_dataframe(sns.scatterplot, x="maxPlayer", y="truecoopmod", color="green")

#g.map_dataframe(plot_ess,  color=defectColor.hex, label="defect investment")
#g.map_dataframe(plot_so,  color=coopColor.hex, label="social optimum investment")
#g.map_dataframe(plot_span)
#g.map(annotator, itera=iter(string.ascii_lowercase))
#g.set(ylabel='Mean investment for $\hat{n}$', xlabel="Number of agents in the environment")
#g.set_titles("$\omega = {col_name} | \hat{{n}} = {row_name}$")

#g.set_xlabel('Number of agents in the environment')
g.add_legend()

In [ ]:
df.query('tau == 10000')

In [ ]:
import json
paths = sorted(glob.glob(f'{prefix}/logs/negociate17-2mut*/2019-12-18-1559/**/rep00/', recursive=True))

allstat = []
for path in tqdm.tqdm_notebook(paths):
    try:
        fitness = read_csvx(path+'/../fit.txt', delimiter="\t", names=['gen', 'min', 'q1', 'med', 'q3', 'max'])
        params = readparams(sorted(glob.glob(path + '/properties*'))[0])
        coop = read_csvx(gethighest(glob.glob(path + '/coop_199.txt'), 'coop_'), delimiter="\t")
        params['coop'] = coop['coop'].mean()
    except Exception as e:
        print('no analysis for',  path)
       # print(type(e), ':', e)
        pass
    else:
        allstat.append(params)
df = pd.DataFrame(allstat)

In [ ]:
df['nbRobots'] = df['gInitialNumberOfRobots']
df['varPheno'] = df['fakeRobots']
df['nbRep'] = df['nbEvaluationsPerGeneration']
df['norm_tau'] = df['tau'] / 100000
df['norm_tau'] = df['norm_tau'].replace(0, 1)

In [ ]:
g = sns.relplot(data=df, x='nbRobots', y='coop', row='mutCoop', col='varPheno', aspect=1.5, kind="line")
g.map_dataframe(sns.scatterplot, x="nbRobots", y="coop")
#g.map_dataframe(sns.scatterplot, x="maxPlayer", y="truecoopmod", color="green")

#g.map_dataframe(plot_ess,  color=defectColor.hex, label="defect investment")
#g.map_dataframe(plot_so,  color=coopColor.hex, label="social optimum investment")
#g.map_dataframe(plot_span)
#g.map(annotator, itera=iter(string.ascii_lowercase))
#g.set(ylabel='Mean investment for $\hat{n}$', xlabel="Number of agents in the environment")
#g.set_titles("$\omega = {col_name} | \hat{{n}} = {row_name}$")

#g.set_xlabel('Number of agents in the environment')
g.add_legend()